In [1]:
# ==============================================================================
# SUIVI DES COTISATIONS CELI (Compte d'épargne libre d'impôt)
# ==============================================================================
# Ce script suit les cotisations CELI et calcule les droits de cotisation restants
# pour l'année fiscale en cours. Il récupère les données de transaction de Disnat
# et compare les cotisations récentes avec la limite annuelle de cotisation.

# ==============================================================================
# IMPORTATIONS ET DÉPENDANCES
# ==============================================================================
# Importer les outils personnalisés pour lire les Google Sheets et les données financières
from bydoux_tools import read_google_sheet_csv
import warnings
import numpy as np
from scipy.optimize import minimize
import bydoux_tools as bt  # Module d'outils financiers personnalisés
from astropy.time import Time, TimeDelta  # Pour les calculs de temps précis
from astropy.table import Table  # Pour les tables de données structurées
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from tqdm import tqdm  # Barres de progression pour les boucles
import matplotlib.cm as cm

# ==============================================================================
# PARAMÈTRES DES COTISATIONS CELI POUR 2025
# ==============================================================================
# Limite annuelle de cotisation CELI telle que publiée par l'Agence du revenu du Canada (ARC)
# Cette valeur doit être mise à jour chaque année lorsque l'ARC annonce la nouvelle limite
# Pour 2025, les droits de cotisation cumulatifs sont de 33 504,55 $
droit_cotisation = 33504.55  # Droits de cotisation CELI totaux disponibles (CAD)

# Définir la date de début pour le suivi des cotisations (généralement le 1er janvier de l'année en cours)
# Cela détermine quelles cotisations comptent pour la limite de l'année en cours
date_cotisation = Time('2025-01-01', format='iso')

# ==============================================================================
# VALIDATION : VÉRIFIER SI LA DATE DE COTISATION CORRESPOND À L'ANNÉE EN COURS
# ==============================================================================
# S'assurer que la date de suivi des cotisations correspond à l'année en cours
# Sinon, avertir l'utilisateur de mettre à jour la limite de cotisation (+7 500 $ pour la nouvelle année)
if date_cotisation.iso[0:4] != Time.now().iso[0:4]:
    bt.printc(f'Année de la date de cotisation ({date_cotisation.iso[0:4]}) différente de l\'année actuelle ({Time.now().iso[0:4]})')
    bt.printc('Vérifiez la date de cotisation et le droit de cotisation et ajouter +7500 aux droits!')

# ==============================================================================
# RÉCUPÉRATION DES DONNÉES : RÉCUPÉRER L'HISTORIQUE DES TRANSACTIONS DE DISNAT
# ==============================================================================
# Obtenir le résumé complet des transactions du compte de courtage Disnat
# Cela inclut tous les types de transactions (achats, ventes, cotisations, etc.)
tbl = bt.get_disnat_summary()

# ==============================================================================
# FILTRAGE DES DONNÉES : ISOLER LES COTISATIONS CELI
# ==============================================================================
# Filtrer la table des transactions pour n'inclure que les cotisations CELI
# 'COTISATION' est le code de type de transaction pour les cotisations CELI dans Disnat
tbl = tbl[tbl['Type de transaction'] == 'COTISATION']

# Trier les transactions par date julienne modifiée (MJD) pour obtenir l'ordre chronologique
# Cela garantit que nous traitons les cotisations dans l'ordre où elles se sont produites
tbl = tbl[np.argsort(tbl['mjd'])]

# ==============================================================================
# CALCUL DES COTISATIONS : SOMMER LES COTISATIONS RÉCENTES
# ==============================================================================
# Créer un masque booléen pour identifier les cotisations effectuées après la date de début de suivi
# Seules les cotisations après date_cotisation comptent pour la limite de l'année en cours
g = tbl['mjd'] > date_cotisation.mjd

# Sommer tous les montants de cotisation qui se sont produits après la date de début de suivi
# "Montant de l'opération" contient le montant de la transaction en CAD
cotisations_recentes = np.sum(tbl["Montant de l'opération"][g])

# ==============================================================================
# FORMATAGE ET AFFICHAGE DES RÉSULTATS
# ==============================================================================
# Formater les montants monétaires en utilisant la fonction de formatage personnalisée
# bt.pdollar() formate les nombres comme des chaînes de devises (ex: "1 234,56 $")
cotisations_txt = bt.pdollar(cotisations_recentes)

# Calculer les droits de cotisation restants en soustrayant les cotisations utilisées de la limite totale
droits_txt = bt.pdollar(droit_cotisation - cotisations_recentes)

# Afficher les résultats en utilisant une sortie colorée pour une meilleure visibilité
# bt.printc() imprime du texte coloré sur la console pour l'emphase
bt.printc(f'Cotisations récentes : {cotisations_txt}')  # Cotisations récentes
bt.printc(f'Droit de cotisation restants : {droits_txt}')  # Droits de cotisation restants

00:03:09.813 |  Cotisations récentes : 15 718,61$
00:03:09.814 |  Droit de cotisation restants : 17 785,94$
